## **📤 Mount your Google Drive before doing anything!**

In [ ]:
#@title Mount Google Drive

# Install Google Drive OCamlfuse
!wget -qO google-drive-ocamlfuse.deb https://launchpad.net/~alessandro-strada/+archive/ubuntu/ppa/+files/google-drive-ocamlfuse_0.7.32-0ubuntu5~bpo22.04.1_amd64.deb
!sudo DEBIAN_FRONTEND=noninteractive apt-get -qq install ./google-drive-ocamlfuse.deb > /dev/null 2>&1
!rm -f google-drive-ocamlfuse.deb
!mkdir -p "/content/drive/MyDrive"
!google-drive-ocamlfuse "/content/drive/MyDrive" -headless -id 822900405000-m241g6dhj59ji3sh7b5dfg2vfa2glkid.apps.googleusercontent.com -secret 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

# 🟢 **Welcome to Fakers-Colab!**

This is a ported version for Google Colab.


# 📝 **-** **Overview**

*   Extraction works in full functionality.
*   Training can be done without preview.
*   Merger works in full functionality.
*   You can import/export your workspace using Google Drive.
*   Import/export and other manipulations with the workspace can be done in the "Manage workspace" block.
*   Free runtimes can run for 6-10 hours. Fakers-Colab creates a backup of your workspace while trianing.
*   Google doesn't favor long-term heavy calculations. Therefore, if you want to train for more than two sessions in a row, use two Google accounts. It is recommended to split your training across 2 accounts, but you can use one Google Drive account to store your workspace.

## 💾 **Install or update from GitLab**

* Install or update directly from GitLab.
* Requirements are automatically installed.

In [ ]:
#@title Install or update from GitLab

Mode = "install" # @param ["install", "update", "remove"]
Branch_name = "master" # @param {type:"string"}

from pathlib import Path
if Mode == 'install':
  branch_cmd = f"-b {Branch_name} " if Branch_name else ""
  # Clone from GitLab - MedicDoesStuff edition
  !git clone {branch_cmd}--depth 1 https://gitlab.com/MedicCodesStuff/FakersLab.git
  data = Path("/usr/lib/python3.10/multiprocessing/resource_tracker.py").read_text().replace('if cache:', 'if False:')
  Path("/usr/lib/python3.10/multiprocessing/resource_tracker.py").write_text(data)
elif Mode == 'update':
  %cd /content/FakersLab
  !git pull
elif Mode == 'remove':
  if input("Are you sure you want to delete FakersLab folder? (y/n)").lower().strip() == "y":
    %cd /content
    %rm -r FakersLab/
    print("Done!")
  else:
    print("Cancelled...")

if Mode != 'remove':
  !pip install numpy==1.23.5
  !pip install -r /content/FakersLab/requirements-colab.txt
  !pip install --upgrade scikit-image
  !pip install colorama
  !pip install tensorflow==2.16.2

  Path("/content/workspace").mkdir(parents=True, exist_ok=True)
  for subdir in ["data_src", "data_src/aligned", "data_dst", "data_dst/aligned", "model"]:
    Path(f"/content/workspace/{subdir}").mkdir(parents=True, exist_ok=True)

print("\nDone!")


## 📁 **Manage workspace**



*   You can import/export the workspace or individual data, such as model files, using Google Drive.
*   You can clear the entire workspace or delete specific parts of it.



In [ ]:
#@title Import from Drive

Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "workspace.zip" #@param {type:"string"}

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name

  %cd $path
  copy_cmd = "/content/drive/MyDrive/"+Archive_name+" "+path
  print("Copying...")
  !cp $copy_cmd
  print("Unzipping...")
  !unzip $unzip_cmd
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")

print("Done!")



In [ ]:
#@title Export to Drive { form-width: "30%" }
Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "merged", "merged_mask", "models", "result video", "result_mask video"]
Archive_name = "workspace.zip" #@param {type:"string"}

def zip_and_copy(path, mode):
  zip_cmd="-0 -r -q "+Archive_name+" "

  %cd $path
  zip_cmd+=mode
  print(f"Zipping...")
  !zip $zip_cmd
  copy_cmd = " "+Archive_name+"  /content/drive/MyDrive/"
  print("Copying...")
  !cp $copy_cmd
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "merged":
  zip_and_copy("/content/workspace/data_dst", "merged")
elif Mode == "merged_mask":
  zip_and_copy("/content/workspace/data_dst", "merged_mask")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
elif Mode == "result video":
  !cp /content/workspace/result.mp4 /content/drive/MyDrive/
elif Mode == "result_mask video":
  !cp /content/workspace/result_mask.mp4 /content/drive/MyDrive/

print("Done!")


In [ ]:
#@title Delete and recreate
Mode = "Delete and recreate workspace" #@param ["Delete and recreate workspace", "Delete models", "Delete data_src", "Delete data_src aligned", "Delete data_src video", "Delete data_dst", "Delete data_dst aligned", "Delete merged frames"]

%cd "/content"

if Mode == "Delete and recreate workspace":
  cmd = "rm -r /content/workspace ; mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"
elif Mode == "Delete models":
  cmd = "rm -r /content/workspace/model/*"
elif Mode == "Delete data_src":
  cmd = "rm /content/workspace/data_src/*.png || rm -r /content/workspace/data_src/*.jpg"
elif Mode == "Delete data_src aligned":
  cmd = "rm -r /content/workspace/data_src/aligned/*"
elif Mode == "Delete data_src video":
  cmd = "rm -r /content/workspace/data_src.*"
elif Mode == "Delete data_dst":
  cmd = "rm /content/workspace/data_dst/*.png || rm /content/workspace/data_dst/*.jpg"
elif Mode == "Delete data_dst aligned":
  cmd = "rm -r /content/workspace/data_dst/aligned/*"
elif Mode == "Delete merged frames":
  cmd = "rm -r /content/workspace/data_dst/merged; rm -r /content/workspace/data_dst/merged_mask"

answer = input("Are you sure you want to " + Mode + "? (y/n)")
while answer not in ["y", "n"]:
  print("Invalid input")
  answer = input("Are you sure you want to " + Mode + "? (y/n)")

if answer == "y":
  !$cmd
  print("Done!")
else:
  print("Cancelled...")

## 🛠️ **Extract, sorting and faceset tools**
* Extract frames from the source or destination video.
* Detect and align faces from frames, with optional debug landmarks.
* Export the workspace to Google Drive after extracting and sorting it manually (in the "Manage workspace" block).
* You can enhance your facesets with the enhancer.
* Resize faceset for model compatibility; saves time during training. Keep the original faceset saved on your PC.
* Pack or unpack facesets with the packing tool.
* Apply or remove a trained XSeg model to the extracted faces.
* It is recommended to use the Generic XSeg model for automatic segmentation.

In [ ]:
#@title Extract frames
Video = "data_src" #@param ["data_src", "data_dst"]

%cd "/content"

cmd = "FakersLab/main.py videoed extract-video"

if Video == "data_dst":
  cmd+= " --input-file workspace/data_dst.* --output-dir workspace/data_dst/"
else:
  cmd+= " --input-file workspace/data_src.* --output-dir workspace/data_src/"

!python $cmd

In [ ]:
#@title Detect faces
Data = "data_src" #@param ["data_src", "data_dst"]
Detector = "S3FD (whole face)" #@param ["S3FD", "S3FD (whole face)"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
dbg = " --output-debug" if Debug else " --no-output-debug"

folder = "workspace/"+Data
folder_aligned = folder+"/aligned"

cmd = "FakersLab/main.py extract --input-dir "+folder+" --output-dir "+folder_aligned
cmd+=" --detector "+detect_type+" --force-gpu-idxs 0"+dbg

if "whole face" in Detector:
  cmd+=" --face-type whole_face"
%cd "/content"
!python $cmd

In [ ]:
#@title Sort aligned
Data = "data_src" #@param ["data_src", "data_dst"]
sort_type = "hist" #@param ["blur", "motion-blur", "face-yaw", "face-pitch", "face-source-rect-size", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final-by-blur", "final-by-size", "absdiff", "final", "final-fast"]

cmd = "FakersLab/main.py sort --input-dir workspace/"+Data+"/aligned --by "+sort_type

%cd "/content"
!python $cmd

In [ ]:
#@title Faceset Enhancer
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/workspace/"+Data+"/aligned"
cmd = "/content/FakersLab/main.py facesettool enhance --input-dir "+data_path
!python $cmd

In [ ]:
#@title Resize faceset
Data = "data_src" #@param ["data_src", "data_dst"]

cmd = "/content/FakersLab/main.py facesettool resize --input-dir /content/workspace/" + \
      f"{Data}/aligned"

!python $cmd

In [ ]:
#@title Pack/Unpack aligned faceset

Folder = "data_src" #@param ["data_src", "data_dst"]
Mode = "unpack" #@param ["pack", "unpack"]

cmd = "/content/FakersLab/main.py util --input-dir /content/workspace/" + \
      f"{Folder}/aligned --{Mode}-faceset"

!python $cmd

In [ ]:
#@title Apply or remove XSeg mask to the faces
Mode = "Apply mask" # @param ["Apply mask", "Remove mask"]
Data = "data_src" # @param ["data_src", "data_dst"]
GenericXSeg = True # @param {type:"boolean"}

from pathlib import Path

mode_arg = 'apply' if Mode == "Apply mask" else 'remove'

if GenericXSeg:
    xseg_path = Path('/content/GenericXSeg')
    if not xseg_path.exists():
        print('Downloading Generic XSeg model ...')
        xseg_url = 'https://huggingface.co/medicreal/FL-Dependencies/resolve/main/model_generic_xseg.zip?download=true'
        xseg_path.mkdir(parents=True)
        !wget -q --no-check-certificate $xseg_url -O /content/GenericXSeg.zip
        !unzip -q /content/GenericXSeg.zip -d /content/GenericXSeg/
        !rm /content/GenericXSeg.zip

main_path = '/content/FakersLab/main.py'
data_path = f'/content/workspace/{Data}/aligned'
model_path = '/content/workspace/model' if not GenericXSeg else str(xseg_path)

cmd = f'python {main_path} xseg {mode_arg} --input-dir {data_path} '
cmd += f'--model-dir {model_path}' if mode_arg == 'apply' else ''

!$cmd

## 🤖 **Train model**

* Choose your model type, but SAEHD is recommended for everyone.
* Set model options in the output field.
* You can manually preview by going to the model folder in the file manager and double-clicking on the PNG files.
* Your workspace will be archived and uploaded to the mounted Drive after 11 hours from the start of the session.
* If you select the "Backup_every_hour" option, your workspace will be backed up every hour.
* You can also manually export your workspace in the "Manage workspace" block.
* The "Silent_Start" option automatically starts with the best GPU and the last model you used.

In [ ]:
#@title Uptime
import psutil, os, time

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()
print(f'Session in running from {int(uptime / 3600)} hours')

In [13]:
#@title Training
Model = "SAEHD" #@param ["SAEHD", "AMP", "AMPLegacy", "Quick96", "XSeg"]
Train_only_src = False #@param {type:"boolean"}
Archive_name_backup = "workspace.zip" #@param {type:"string"}
Pretraining_dataset_path = '/content/pretrain/' #@param {type:"string"}
Export_model_only = False #@param {type:"boolean"}
Backup_every_specified_time = True #@param {type:"boolean"}
backup_time = 1 #@param {type:"number"}
Silent_Start = True #@param {type:"boolean"}

%cd "/content"

import psutil, os, time

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

if Export_model_only:
  if os.path.exists(Archive_name_backup) and not os.path.exists(Archive_name_backup.replace('.zip', '_model.zip')):
    os.system(f"rm {Archive_name_backup}")
    print('Full workspace archive deleted\nCreating model archive...')
    os.system('zip -0 -r -q '+Archive_name_backup.replace('.zip', '_model.zip')+' workspace/model')
    print("Archive created!")
  elif not os.path.exists(Archive_name_backup) and not os.path.exists(Archive_name_backup.replace('.zip', '_model.zip')):
    print('Creating model archive...')
    os.system('zip -0 -r -q '+Archive_name_backup.replace('.zip', '_model.zip')+' workspace/model')
  else:
    print('Archive exists')


if Backup_every_specified_time and not Export_model_only:
  if not os.path.exists(Archive_name_backup):
    if os.path.exists(Archive_name_backup.replace('.zip', '_model.zip')):
      print('Removing model archive...')
      os.system(f"rm {Archive_name_backup.replace('.zip', '_model.zip')}")
    print("Creating workspace archive ...")
    os.system('zip -0 -r -q '+Archive_name_backup+' workspace')
    print("Archive created!")
  else:
    print("Archive exists!")

if Backup_every_specified_time:
  print("Time to end session: "+str(round((43200-uptime)/3600))+" hours")
  backup_time = str(backup_time * 3600)
  if not Export_model_only:
    backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -0 -r -q "+Archive_name_backup+" workspace/model'); os.system('cp /content/"+Archive_name_backup+" /content/drive/MyDrive/'); print('Backed up!') \""
  else:
    backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -0 -r -q "+Archive_name_backup.replace('.zip', '_model.zip')+" workspace/model'); os.system('cp /content/"+Archive_name_backup.replace('.zip', '_model.zip') +" /content/drive/My\ Drive/'); print('Backed up!') \""
elif (round(39600-uptime) > 0):
  print("Time to backup: "+str(round((39600-uptime)/3600))+" hours")
  backup_time = str(round(39600-uptime))
  if not Export_model_only:
    backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -0 -r -q "+Archive_name_backup+" workspace'); os.system('cp /content/"+Archive_name_backup+" /content/drive/MyDrive/'); print('Backed up!') \""
  else:
    backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -0 -r -q "+Archive_name_backup.replace('.zip', '_model.zip')+" workspace/model'); os.system('cp /content/"+Archive_name_backup.replace('.zip', '_model.zip') +" /content/drive/My\ Drive/'); print('Backed up!') \""
else:
  print("Session expires in less than an hour.")
  backup_cmd = ""

if Train_only_src:
  if Model not in ["AMP", "AMPLegacy"]:
    answer = ''
    print('The src-src training is only for AMP model. Do you want to continue anyway? (y/n)')
    while True:
      answer = input()
      if answer == 'y':
        cmd = "FakersLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_src/aligned --pretraining-data-dir " + Pretraining_dataset_path + " --model-dir workspace/model --model "+Model
      elif answer == 'n':
        cmd = "FakersLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir " + Pretraining_dataset_path + " --model-dir workspace/model --model "+Model
      else:
        print('Wrong value! Insert y or n:')
      if answer in ['y', 'n']:
        break
  else:
    cmd = "FakersLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_src/aligned --pretraining-data-dir " + Pretraining_dataset_path + " --model-dir workspace/model --model "+Model
else:
  cmd = "FakersLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir " + Pretraining_dataset_path + " --model-dir workspace/model --model "+Model

if Model == "Quick96":
  cmd += " --pretrained-model-dir pretrain_Q96"

if Silent_Start:
  cmd += " --silent-start"

if (backup_cmd != ""):
  train_cmd = (cmd+backup_cmd)
else:
  train_cmd = (cmd)

!python $train_cmd

/content
Time to backup: 11 hours
                                                               #####                                                                                       .##############*                                                                                      ###############(###                                                               **           ##*#########/(############                                                            /&&&&%*      ######### ##   (# #######                                                              /&&&&&&&&&/, (####### #       #*######                                                              *&&&&&&&&&&&&&#/#####.#(      # #########                                                           *&&&&&&&&&&&&&%%%%%((## /#### ##########                                                     ###    *&%%%%%%%%&&&&%%%%%%%%%#(###########                                                     ########(##(%%%%%%%%&&&&&&&&&&&%%%%%%%#

In [14]:
!wget https://images.gmanews.tv/webpics/2024/07/Alice_Guo_with_lawyer_2024_07_15_21_32_07.jpg

--2024-08-23 06:41:57--  https://images.gmanews.tv/webpics/2024/07/Alice_Guo_with_lawyer_2024_07_15_21_32_07.jpg
Resolving images.gmanews.tv (images.gmanews.tv)... 54.230.176.109, 54.230.176.79, 54.230.176.59, ...
Connecting to images.gmanews.tv (images.gmanews.tv)|54.230.176.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48921 (48K) [image/jpeg]
Saving to: ‘Alice_Guo_with_lawyer_2024_07_15_21_32_07.jpg’

Alice_Guo_with_lawy 100%[===================>]  47.77K   287KB/s    in 0.2s    

2024-08-23 06:41:59 (287 KB/s) - ‘Alice_Guo_with_lawyer_2024_07_15_21_32_07.jpg’ saved [48921/48921]



In [6]:
!pip install colorama

## ⚪ **Merge frames**

In [ ]:
#@title Merge
Model = "AMP" #@param ["SAEHD", "AMP", "Quick96"]

cmd = "FakersLab/main.py merge --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --output-mask-dir workspace/data_dst/merged_mask --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [ ]:
#@title Get result video
Mode = "result video" #@param ["result video", "result_mask video"]
Bitrate = 24 #@param {type:"integer"}
Copy_to_Drive = True #@param {type:"boolean"}

cmd = "FakersLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4 --include-audio --bitrate " + str(Bitrate)
cmd_mask = "FakersLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged_mask --output-file workspace/result_mask.mp4 --reference-file workspace/data_dst.mp4 --bitrate " +str(Bitrate)

if Mode == "result video":
  !python $cmd
  if Copy_to_Drive:
    !cp /content/workspace/result.mp4 /content/drive/MyDrive/
elif Mode == "result_mask video":
  !python $cmd_mask
  if Copy_to_Drive:
    !cp /content/workspace/result_mask.mp4 /content/drive/MyDrive/
